In [249]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [250]:
from moffragmentor import MOF
from moffragmentor.fragmentor.solventlocator import get_floating_solvent_molecules, get_all_bound_solvent_molecules
from moffragmentor.fragmentor.nodelocator import find_node_clusters, create_node_collection
from moffragmentor.fragmentor.linkerlocator import create_linker_collection
from moffragmentor.utils import visualize_part, _get_metal_sublist, unwrap,_not_relevant_structure_indices
from moffragmentor.fragmentor.filter import point_in_mol_coords

In [251]:
mof = MOF.from_cif('/Users/kevinmaikjablonka/git/kjappelbaum/moffragmentor/tests/test_files/MUDLON_clean.cif')
#mof = MOF.from_cif('/Users/kevinmaikjablonka/git/kjappelbaum/moffragmentor/tests/test_files/porphyrin_mof.cif')

In [217]:
unbound_solvent = get_floating_solvent_molecules(mof)
    # Find nodes
node_result = find_node_clusters(mof, unbound_solvent.indices)
node_collection = create_node_collection(mof, node_result)
    # Find bound solvent
    # 
bound_solvent = get_all_bound_solvent_molecules(mof, node_result.nodes)
    # Filter the linkers (valid linkers have at least two branch points)
linker_collection, edge_dict = create_linker_collection(
        mof, node_result, node_collection, unbound_solvent
    )

In [218]:
linker_collection[0].show_molecule()

NGLWidget()

In [198]:
metal_in_node = _get_metal_sublist(node_result.nodes[0], mof.metal_indices)

In [199]:
in_hull( unwrap(mof.cart_coords[linker_collection[0]._original_indices], mof.lattice), mof.cart_coords[metal_in_node[0]])

False

In [200]:
point_in_mol_coords(mof.cart_coords[metal_in_node[0]], 
  mof.cart_coords[linker_collection[2]._original_indices], mof.lattice)

IndexError: list index out of range

In [201]:
linker_collection[2].show_molecule()

IndexError: list index out of range

In [245]:
visualize_part(mof, [node_collection[3]._original_indices[2]])

NGLWidget()

In [154]:
sites = []
for index in linker_collection[2]._original_indices + [0]:
    sites.append(mof.structure[index])

s = Structure.from_sites(sites, to_unit_cell=True)

s.to('cif', 'bla.cif')

In [235]:
node_collection[3]._original_indices

[193, 290, 3, 291, 226, 294, 295, 296, 297, 206, 285, 284, 253]

In [248]:
len(mols)

0

In [184]:
import numpy as np
import pandas as pd

coords = np.vstack([mof.cart_coords[metal_in_node[1]].reshape(1,3), unwrap(mof.cart_coords[linker_collection[0]._original_indices], mof.lattice) ] )

IndexError: list index out of range

In [183]:
df = pd.DataFrame(data=coords, columns=['x', 'y', 'z'])

In [178]:
df['color'] = ['red'] + ['blue'] *(len(df) -1)

In [179]:
import plotly.express as px

fig = px.scatter_3d(df, x='x', y='y', z='z',
              color='color', opacity=0.7)
fig

In [31]:
from pymatgen.core import Structure

sites = []
for index in linker_collection[0]._original_indices + [metal_in_node[0]]:
    sites.append(mof.structure[index])

s = Structure.from_sites(sites)
s.to('cif', 'mudon_linker.cif')

In [252]:
bbs = mof.fragment()

2022-04-14 10:44:32.166 | INFO     | moffragmentor.fragmentor:run_fragmentation:45 - Checking for metal in linker
2022-04-14 10:44:34.105 | INFO     | moffragmentor.fragmentor:run_fragmentation:93 - Constructing the embedding


In [253]:
bbs.linkers[0].show_molecule()

NGLWidget()